In [8]:
import pandas as pd
import geopandas as gpd 
import matplotlib.pyplot as plt
import folium 
from shapely.ops import nearest_points
from shapely.geometry import LineString

In [9]:
covid=pd.read_csv("time_series_covid19_deaths_global_narrow.csv")
covid = covid.set_index('Date')
covid.head()

,Province/State,Country/Region,Lat,Long,Value,ISO 3166-1 Alpha 3-Codes,Region Code,Sub-region Code,Intermediate Region Code
Date,,,,,,,,,
4/27/2020,NaN,Afghanistan,33.0,65.0,57,AFG,142.0,34.0,NaN
4/26/2020,NaN,Afghanistan,33.0,65.0,50,AFG,142.0,34.0,NaN
4/25/2020,NaN,Afghanistan,33.0,65.0,47,AFG,142.0,34.0,NaN
4/24/2020,NaN,Afghanistan,33.0,65.0,43,AFG,142.0,34.0,NaN
4/23/2020,NaN,Afghanistan,33.0,65.0,42,AFG,142.0,34.0,NaN


In [11]:
air=pd.read_csv("../2020_combined.csv")
air=air.set_index('Date')
air=air.drop(columns = ['Unnamed: 0'])
air.head()

,Country,City,Specie,count,min,max,median,variance,Pop,Lat,Long
Date,,,,,,,,,,,
1/4/2020,PR,San Juan,pm25,24,15.0,56.0,24.0,1351.23,418140,18.46633,-66.10572
1/7/2020,PR,San Juan,pm25,5,20.0,25.0,22.0,63.00,418140,18.46633,-66.10572
2/28/2020,PR,San Juan,pm25,8,3.0,102.0,43.0,9862.68,418140,18.46633,-66.10572
3/3/2020,PR,San Juan,pm25,5,46.0,158.0,59.0,20923.00,418140,18.46633,-66.10572
3/7/2020,PR,San Juan,pm25,6,3.0,176.0,33.0,44312.00,418140,18.46633,-66.10572


In [12]:
def create_gdf(df, x="Lat", y="Long"):
    return gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[x], df[y]), crs={"init":"epsg:4326"})

In [13]:
covid_gdf = create_gdf(covid)
air_gdf = create_gdf(air)

In [14]:
def calculate_nearest(row, destination, val, col="geometry"):
    dest_unary = destination["geometry"].unary_union
    nearest_geom = nearest_points(row[col], dest_unary)
    match_geom = destination.loc[destination.geometry == nearest_geom[1]]
    match_value = match_geom[val].to_numpy()[0]
    return match_value

In [15]:
air_gdf["Region"] = air_gdf.apply(calculate_nearest, destination=covid_gdf, val="Province/State", axis=1)

KeyboardInterrupt: 

In [ ]:
air_gdf["CCountry"] = air_gdf.apply(calculate_nearest, destination=covid_gdf, val="Country/Region", axis=1)

In [ ]:
air_gdf["Deaths"] = air_gdf.apply(calculate_nearest, destination=covid_gdf, val="Value", axis=1)

In [ ]:
air_gdf.head()